In [1]:
images = {
            'company_logo': 'static/images/company_logo.png',
            'phone_icon': 'static/images/phone_icon.png',
            'email_icon': 'static/images/email_icon.png',
            'web_icon': 'static/images/web_icon.png'
        }

print(images.items())

dict_items([('company_logo', 'static/images/company_logo.png'), ('phone_icon', 'static/images/phone_icon.png'), ('email_icon', 'static/images/email_icon.png'), ('web_icon', 'static/images/web_icon.png')])
